<a href="https://colab.research.google.com/github/vijayrgopu/apache_beam/blob/master/12a_PubSub_FixedWindow_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --quiet apache_beam[gcp]

In [0]:
from google.colab import files
uploaded = files.upload()

Saving vg-proj1-pub-sub-4eb0685e1c86.json to vg-proj1-pub-sub-4eb0685e1c86.json


In [0]:
import os
import time
from apache_beam import window
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.transforms.trigger import AfterWatermark, AfterProcessingTime, AfterCount, AccumulationMode, Repeatedly


subscription = 'projects/vg-proj1-pub-sub/subscriptions/sub1-top1-vg-proj1-pub-sub'
push_to_topic = 'projects/vg-proj1-pub-sub/topics/top2-vg-proj1-pub-sub'

os.environ['GOOGLE_APPLICATION_CREDENTIALS']='vg-proj1-pub-sub-4eb0685e1c86.json'

options = PipelineOptions()
options.view_as(StandardOptions).streaming = True

class Strip_Chars(beam.DoFn):
  def process(self,in_text):
    print('returning {}'.format(in_text.rstrip().lstrip().decode()))
    return in_text.rstrip().lstrip().decode()

class Split_To_Columns(beam.DoFn):
  def __init__(self,delimiter=','):
    self.delimiter = delimiter
  def process(self,line):
    #print('splitting {}'.format(lambda line : line.split(self.delimiter)))
    return line 

def filter_city(elements):
  print('Elements List {}'.format(elements))
  return elements


def calculate_profit(elements):
  str_id = elements[0]
  count = int(elements[4])
  cost_price = int(elements[5])
  sell_price = int(elements[6])
  profit = (sell_price - cost_price)*count
  elements.append(str(profit))
  print('elements = {}'.format(elements))
  return elements

#setting eventtime and letting beam know  
def custom_timestamp(elements):
  unix_epoch = elements[7]
  return beam.window.TimestampedValue(elements,int(unix_epoch))

def encode_byte_string(elements):
  element  = str(elements)
  return element.encode('utf-8')

p1 = beam.Pipeline(options=options)

read_messages = (
    p1
    |'Read from sub1' >> beam.io.ReadFromPubSub(subscription=subscription)
    |'Remove extra chars' >> beam.Map(lambda data: (data.rstrip().lstrip()).decode())           # STR_2,Mumbai,PR_265,Cosmetics,8,39,66,
    #|'Removing extra Chars' >> beam.ParDo(Strip_Chars())
    |'Splitting rows' >> beam.Map(lambda line : line.split(','))
    | 'Filter By City' >> beam.Filter(lambda elements : (elements[1] == 'Mumbai' or elements[1] == 'Bangalore'))
    |'Calculate Profit' >> beam.Map(calculate_profit)    
    #|'apply event timestamp' >> beam.Map(custom_timestamp) #Code to add the eventtimestamp and let Beam know the event timestamp
    #|'From Key value Pair' >> beam.Map(lambda elements : (elements[3],int(elements[4]))) #Session Window
    |'From Key value Pair' >> beam.Map(lambda elements : (elements[0],int(elements[8]))) #For fixed and sliding window
    #|'set processing Window' >> beam.WindowInto(window.FixedWindows(20))
    #|'set processing Window' >> beam.WindowInto(window.SlidingWindows(30,10))
    #|'set processing Window' >> beam.WindowInto(window.GlobalWindows(),trigger=Repeatedly(AfterCount(5)),accumulation_mode=AccumulationMode.DISCARDING) # Global Window
    |'set processing Window' >> beam.WindowInto(window.FixedWindows(),trigger=AfterProcessingTime(20),accumulation_mode=AccumulationMode.DISCARDING) # After Processing Time trigger with AccumulationMode.ACCUMULATING
    #|'set processing Window' >> beam.WindowInto(window.Sessions(20))
    |'Calculate Sum' >> beam.CombinePerKey(sum)
    |'Encode to byte string' >> beam.Map(encode_byte_string)
    |'Print out' >> beam.Map(filter_city)
    |'Push to Pubsub' >> beam.io.WriteToPubSub(topic=push_to_topic)
)

result = p1.run()
result.wait_until_finish()

In [0]:
!ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Apr  3 16:24 sample_data
-rw-r--r-- 1 root root 2341 Apr  8 18:01 vg-proj1-pub-sub-4eb0685e1c86.json
